## Introduction to PyTorch

In [1]:
import torch

print(torch.__version__)
torch.cuda.is_available() # Check if GPU is available

1.12.0+cu113


True

### 1. `help()` & `dir()`

`pytorch`(a package) 就像一个工具箱, 有不同的分隔区, 分隔区里有不同的工具.探索工具箱, 两个道具:

- dir() 函数：能让我们知道工具箱以及工具箱中的分隔区有什么东西(打开，看见)
- help() 函数：能让我们知道每个工具是如何使用的，工具的使用方法(说明书)

In [5]:
print(dir(torch.cuda)[:10])
print("_____________________________")
help(torch.cuda.Any)


['Any', 'BFloat16Storage', 'BFloat16Tensor', 'BoolStorage', 'BoolTensor', 'ByteStorage', 'ByteTensor', 'CUDAGraph', 'CharStorage', 'CharTensor']
_____________________________
Help on _SpecialForm in module typing:

typing.Any
    Special type indicating an unconstrained type.
    
    - Any is compatible with every type.
    - Any assumed to have all methods.
    - All values assumed to be instances of Any.
    
    Note that all the above statements are true from the point of view of
    static type checkers. At runtime, Any should not be used with instance
    or class checks.



### 2. DataLoad

PyTorch 读取数据涉及两个类: `Dataset` & `Dataloader`

#### 2.1 `Dataset`

`Dataset`: 提供一种方式, 获取其中需要的数据及其对应的真实的 label 值, 并完成编号. 主要实现以下两个功能:

- 如何获取每一个数据及其label
- 告诉我们总共有多少的数据

`Dataset` 是一个**抽象类**, 所有**数据集**都需要继承这个类, 我们需要将要使用的数据包装为Dataset类. 所有子类都需要重写 `__getitem__` 的方法, 这个方法主要是获取每个数据集及其对应 label, 还可以重写长度类` __len__`


#### 2.2 `Dataloader`
`Dataloader`: 打包(batch_size), 为后面的神经网络提供不同的数据形式

- **shuffle**: 当 shuffle 设置为 True 时， DataLoader 会在每个 **epoch** 开始时随机打乱数据集中的样本. 这有助于提高模型的泛化能力, 防止模型在训练过程中对数据顺序的记忆.
- **num_workers**: 多少个进程来读取数据

In [3]:
import torch
from torch.utils.data import Dataset, DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
import torchvision
from PIL import Image   #读取图片
import os 

In [ ]:

#创建一个class，继承 Dataset 类
class MyData(Dataset):
  def __init__(self,root_dir,label_dir):  
    
    #通过索引获取图片的地址，需要先创建图片地址的list
    self.root_dir=root_dir    
    self.label_dir=label_dir
    self.path=os.path.join(self.root_dir,self.label_dir)
    self.img_path=os.listdir(self.path)  #获得图片下所有的地址
    self.transform = torchvision.transforms.Compose([
        torchvision.transforms.Resize((128, 128)),  #将图片resize到128*128
        torchvision.transforms.ToTensor(),  #将图片转换为tensor
    ])

  def __getitem__(self, idx):   #idx为编号
    #获取每一个图片
    img_name=self.img_path[idx]  #名称
    img_item_path = os.path.join(self.root_dir,self.label_dir,img_name)  # 每张图片的相对路径
    img = Image.open(img_item_path).convert("RGB")  # 确保图像为RGB格式
    img = self.transform(img)
    label=self.label_dir
    return img,label
 
  def __len__(self):    #数据集的长度
    return len(self.img_path)
 
 
#用类创建实例
root_dir="../Data/hymenoptera_data/train"
ants_label_dir="ants"
bees_label_dir="bees"
ants_dataset=MyData(root_dir,ants_label_dir)
bees_dataset=MyData(root_dir,bees_label_dir)
 
 
#将ants(124张)和bees(121张)两个数据集进行拼接
train_dataset = ants_dataset+bees_dataset


img, target = train_dataset[0]
print(img.shape)
print(target)

torch.Size([3, 128, 128])
ants


In [ ]:
# 创建 DataLoader
train_loader = DataLoader(dataset=train_dataset, batch_size=16, shuffle=True, drop_last=False)

# 获取迭代器
# iter(...) 创建一个迭代器，用于遍历 DataLoader 中的批次

data_iter = iter(train_loader)

# next(...) 从迭代器中获取下一个批次的数据，返回一个元组 (X, y)，其中 X 是图像张量，y 是对应的标签
# 获取第一个 batch
first_batch = next(data_iter)

# 获取第二个 batch
second_batch = next(data_iter)
inputs, targets = second_batch

print("输入数据形状:", inputs.shape)
print(targets)

输入数据形状: torch.Size([16, 3, 128, 128])
('ants', 'bees', 'ants', 'ants', 'bees', 'bees', 'ants', 'bees', 'bees', 'bees', 'bees', 'ants', 'bees', 'ants', 'bees', 'ants')


#### Another way to build dataloader

In [12]:
# 创建一个大小为 (700, 1, 28, 28) 的随机矩阵, 代表700张单通道28*28像素的照片
X = torch.randn(700, 1, 28, 28)
# X = X.reshape(700, 28, 28)
# 生成 700 个随机标签，范围从 1 到 10
y = torch.randint(1, 11, (700,))  

# Flatten images, create train-test split
#X_flat = X.reshape(700, 28 * 28)
#X_train, X_test, y_train, y_test = train_test_split(X_flat, y, stratify=y)
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y)

# Create PyTorch datasets
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

# Create DataLoaders
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, num_workers=4, shuffle=True, pin_memory=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, num_workers=4, shuffle=False, pin_memory=False)

In [13]:
data_iter = iter(train_loader)
first_batch = next(data_iter)
first_batch[0].size()

torch.Size([32, 1, 28, 28])